In [ ]:
# pip install transformers accelerate qwen-vl-utils==0.0.8 torch torchvision
# pip install git+https://github.com/jacobgil/pytorch-grad-cam.git

In [ ]:
%%capture
# !pip install unsloth
# !pip uninstall unsloth -y && pip install --upgrade --no-cache-dir --no-deps git+https://github.com/unslothai/unsloth.git

# %%capture
# Normally using pip install unsloth (the above code) is enough
# Temporarily as of Jan 31st 2025, Colab has some issues with Pytorch
# Using pip install unsloth will take 3 minutes, whilst the below takes <1 minute:
!pip install --no-deps bitsandbytes accelerate xformers==0.0.29 peft trl triton
!pip install --no-deps cut_cross_entropy unsloth_zoo
!pip install sentencepiece protobuf datasets huggingface_hub hf_transfer
!pip install --no-deps unsloth

!pip uninstall -y transformers
!pip install git+https://github.com/huggingface/transformers.git@1931a351408dbd1d0e2c4d6d7ee0eb5e8807d7bf
# there are some issues with using the qwen VL 3B model: https://github.com/huggingface/transformers/issues/36193
# so i have to uninstall and reinstall the transformers above
# might change to new model later on. update: i am using the official qwen model instead

In [ ]:
!pip install qwen-vl-utils

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 39.7/39.7 MB 15.0 MB/s eta 0:00:00


In [ ]:
from unsloth import FastVisionModel
from transformers import AutoProcessor
from qwen_vl_utils import process_vision_info
import torch

model, tokenizer = FastVisionModel.from_pretrained(
    "Qwen/Qwen2-VL-7B-Instruct",
    load_in_4bit=True,
    use_gradient_checkpointing="unsloth"
)
processor = AutoProcessor.from_pretrained("Qwen/Qwen2-VL-7B-Instruct")

==((====))==  Unsloth 2025.2.15: Fast Qwen2_Vl vision patching. Transformers: 4.49.0.
   \\   /|    GPU: Tesla T4. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = None. FA2 = False]
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


In [ ]:
# pip install --upgrade transformers

In [ ]:
# upload an image with "im.jpeg" name on google colab or change the image_path.
image_path = "im.jpeg"
question = "What is shown in this image?"

messages = [
    {
        "role": "user",
        "content": [
            {"type": "image", "image": image_path},
            {"type": "text", "text": question},
        ],
    }
]

In [ ]:
text = processor.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
image_inputs, video_inputs = process_vision_info(messages)
inputs = processor(
    text=[text],
    images=image_inputs,
    videos=video_inputs,
    padding=True,
    return_tensors="pt"
).to('cuda')

In [ ]:
max_new_tokens = 10
attention_maps = []

for _ in range(max_new_tokens):
    with torch.no_grad():
        outputs = model(**inputs)

    next_token = outputs.logits[:, -1, :].argmax(dim=-1)
    attention_maps.append(outputs.attentions)

    # if next_token.item() == tokenizer.eos_token_id:
    #     break

    inputs['input_ids'] = torch.cat([inputs['input_ids'], next_token.unsqueeze(0)], dim=-1)
    inputs['attention_mask'] = torch.cat([inputs['attention_mask'], torch.ones_like(next_token).unsqueeze(0)], dim=-1)

generated_text = tokenizer.decode(inputs['input_ids'][0], skip_special_tokens=True)
answer = generated_text.split("Answer: ")[-1].strip()

In [ ]:
print(answer)

system
You are a helpful assistant.
user
What is shown in this image?
assistant
The image shows a person standing outdoors in a scenic


In [ ]:
outputs.logits.shape

torch.Size([1, 1036, 152064])

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

def visualize_attention(attention_map, word, layer_idx):
    # Assuming the last token's attention is what we want
    att_map = attention_map[-1][0, -1].mean(dim=0).cpu().detach().numpy()
    plt.figure(figsize=(10, 10))
    plt.imshow(att_map, cmap='viridis')
    plt.title(f"Attention Map - Layer {layer_idx}, Word: {word}")
    plt.colorbar()
    plt.show()

answer_words = answer.split()
for word_idx, word in enumerate(answer_words):
    print(f"Visualizing attention for word: {word}")
    for layer_idx, layer_attention in enumerate(attention_maps[word_idx]):
        visualize_attention(layer_attention, word, layer_idx)


Visualizing attention for word: system


TypeError: 'NoneType' object is not iterable